In [1]:
#Importando bibliotecas
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

import folium
import requests
import pandas as pd
import numpy as np

In [2]:
#Pegando dados da Wikipedia
url  = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
if page.status_code == 200:
    print('Page download successful')
else:
    print('Page download error. Error code: {}'.format(page.status_code))

Page download successful


In [3]:
df_html = pd.read_html(url, header=0, na_values = ['Not assigned'])[0]
df_html.head(10)

,Postcode,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,NaN,NaN
9,M9A,Queen's Park,NaN


In [4]:
#Clean Data
#Excluindo as linhas onde o Borough é Not assigned.
df_html.dropna(subset=['Borough'], inplace=True)
df_html.head(10)

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,NaN
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [5]:
#Para as Neihgborhoods que estão como NaN damos o nome do Borough
df_html['Neighborhood'].fillna(df_html['Borough'], inplace=True)
df_html.head(10)

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [6]:
#rows will be same postalcode will combined into one row with the neighborhoods separated with a comma
result = df_html.groupby(['Postcode','Borough'], sort=False).agg( ', '.join)
dfNew=result.reset_index()
dfNew.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [7]:
print('The shape of the dataset is:',dfNew.shape)

The shape of the dataset is: (103, 3)


In [8]:
# Salvando o Dataframe em um arquivo .csv
dfNew.to_csv('Toronto_Postcodes.csv')

In [9]:
# Pegando as coordendadas geográficas de cada Postalcode
url_csv = 'http://cocl.us/Geospatial_data'
df_coordinates = pd.read_csv(url_csv)
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
#Recuperando os dados Postais em um novo Dataframe
df_neighborhoods = pd.read_csv('Toronto_Postcodes.csv',index_col=[0])
df_neighborhoods.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [11]:
#Padronizando o nome da coluna em ambos os DataFrames
df_coordinates.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_neighborhoods.rename(columns={'Postcode': 'PostalCode'}, inplace=True)

In [12]:
#Juntando ambos os dataframes em um único
df_neighborhoods_coordinates = pd.merge(df_neighborhoods, df_coordinates, on='PostalCode')
df_neighborhoods_coordinates.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


In [13]:
# Pegando as Coordenadas de Toronto
from geopy.geocoders import Nominatim 

address = 'Toronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
lat_toronto = location.latitude
lng_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(lat_toronto, lng_toronto))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [14]:
# Mostrando o mapa de toronto com todos os bairros marcados
map_toronto = folium.Map(location=[lat_toronto, lng_toronto], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_neighborhoods_coordinates['Latitude'], 
                                           df_neighborhoods_coordinates['Longitude'],
                                           df_neighborhoods_coordinates['Borough'], 
                                           df_neighborhoods_coordinates['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [15]:
# Criando credencial para conectar ao Foursquare

CLIENT_ID = 'KV1ZTVX03RWE1METG3FCOKUL2BOVNTMGK5CHOD5TOTH1CUSR' # your Foursquare ID
CLIENT_SECRET = 'HZDYU2YPKXSASI4N1J2ZUUVLODQF4FT2XSMY1Z5NXHV0BZLH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KV1ZTVX03RWE1METG3FCOKUL2BOVNTMGK5CHOD5TOTH1CUSR
CLIENT_SECRET:HZDYU2YPKXSASI4N1J2ZUUVLODQF4FT2XSMY1Z5NXHV0BZLH


In [16]:
# Conectando ao Foursquare e recebendo os lugares
#LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        #url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&raduys={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
   
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
toronto_venues = getNearbyVenues(names=df_neighborhoods_coordinates['Neighborhood'],
                                latitudes=df_neighborhoods_coordinates['Latitude'],
                                longitudes=df_neighborhoods_coordinates['Longitude'])

Parkwoods
Victoria Village
Harbourfront
Lawrence Heights, Lawrence Manor
Queen's Park
Queen's Park
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The Danforth West, Riv

In [18]:
print(toronto_venues.shape)
toronto_venues.head(10)

(3090, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
2,Parkwoods,43.753259,-79.329656,Galleria Supermarket,43.753520,-79.349518,Supermarket
3,Parkwoods,43.753259,-79.329656,Graydon Hall Manor,43.763923,-79.342961,Event Space
4,Parkwoods,43.753259,-79.329656,Island Foods,43.745866,-79.346035,Caribbean Restaurant
5,Parkwoods,43.753259,-79.329656,Darband Restaurant,43.755194,-79.348498,Middle Eastern Restaurant
6,Parkwoods,43.753259,-79.329656,LA Fitness,43.747665,-79.347077,Gym / Fitness Center
7,Parkwoods,43.753259,-79.329656,Me Va Me Kitchen Express,43.754957,-79.351894,Mediterranean Restaurant
8,Parkwoods,43.753259,-79.329656,North Beach Indoor Volleyball Academy,43.737191,-79.323714,Gym / Fitness Center
9,Parkwoods,43.753259,-79.329656,LCBO,43.757774,-79.314257,Liquor Store


In [19]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",30,30,30,30,30,30
Agincourt,30,30,30,30,30,30
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",30,30,30,30,30,30
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",30,30,30,30,30,30
"Alderwood, Long Branch",30,30,30,30,30,30
"Bathurst Manor, Downsview North, Wilson Heights",30,30,30,30,30,30
Bayview Village,30,30,30,30,30,30
"Bedford Park, Lawrence Manor East",30,30,30,30,30,30
Berczy Park,30,30,30,30,30,30


In [20]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
toronto_venues['Venue Category'].unique()[:100]

There are 254 uniques categories.


array(['Caribbean Restaurant', 'Golf Course', 'Supermarket',
       'Event Space', 'Middle Eastern Restaurant', 'Gym / Fitness Center',
       'Mediterranean Restaurant', 'Liquor Store', 'Coffee Shop', 'Café',
       'Italian Restaurant', 'Park', 'Shopping Mall', 'Bagel Shop',
       'Greek Restaurant', 'Seafood Restaurant', 'Movie Theater',
       'Japanese Restaurant', 'Ice Cream Shop', 'Grocery Store',
       'Persian Restaurant', 'Restaurant', 'Thai Restaurant',
       'History Museum', 'Indian Restaurant', 'Hockey Arena',
       'Rock Climbing Spot', 'Brewery', 'Cosmetics Shop', 'Bakery',
       'Sandwich Place', 'Gastropub', 'Concert Hall', 'Science Museum',
       'Turkish Restaurant', 'Historic Site', 'Farmers Market', 'Spa',
       'Chocolate Shop', 'Dessert Shop', 'Performing Arts Venue', 'Pub',
       'French Restaurant', 'Breakfast Spot', 'Tech Startup',
       'Mexican Restaurant', 'Boutique', 'Burger Joint', 'Pharmacy',
       'Furniture / Home Store', 'Clothing Store', '

In [21]:
# Verificar se "Brazilian Restaurant" estão entre as venues
"Brazilian Restaurant" in toronto_venues['Venue Category'].unique()

True

In [22]:
#Criar um novo Dataframe mostrando a quatidade de cada Venue Category em cada Neighborhoods
#Manipulating the data to make the analysis easy

to_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(3090, 254)


,Zoo Exhibit,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
to_grouped = to_onehot.groupby('Neighborhood').mean().reset_index()

print(to_grouped.shape)
to_grouped

(102, 254)


,Neighborhood,Zoo Exhibit,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Aquarium,Art Gallery,Arts & Crafts Store,...,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo
0,"Adelaide, King, Richmond",0.0,0.000000,0.000000,0.000000,0.033333,0.000000,0.0,0.000000,0.000000,...,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.033333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.066667,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Alderwood, Long Branch",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
8,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.033333,0.000000,...,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [24]:
# DATA ANALYSIS

In [25]:
len(to_grouped[to_grouped['Brazilian Restaurant'] > 0])

3

In [26]:
to_brazilian = to_grouped[['Neighborhood','Brazilian Restaurant']]
to_brazilian

,Neighborhood,Brazilian Restaurant
0,"Adelaide, King, Richmond",0.000000
1,Agincourt,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000
4,"Alderwood, Long Branch",0.000000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000
6,Bayview Village,0.000000
7,"Bedford Park, Lawrence Manor East",0.000000
8,Berczy Park,0.000000
9,"Birch Cliff, Cliffside West",0.000000


In [29]:
toronto_merged = pd.merge(df_neighborhoods_coordinates, to_brazilian, on='Neighborhood')
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Brazilian Restaurant
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,0.0
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0.0


In [30]:
# Pegar apenas bairros que tenham restaurantes Brasileiros               
df_remove = toronto_merged.loc[(toronto_merged['Brazilian Restaurant'] <= 0)]
restaurant_br_df = toronto_merged.drop(df_remove.index)
restaurant_br_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Brazilian Restaurant
21,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512,0.066667
31,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259,0.033333
56,M6M,York,"Del Ray, Keelesdale, Mount Dennis, Silverthorn",43.691116,-79.476013,0.066667


In [31]:
# Mostrando o mapa de toronto com todos os bairros marcados
map_brazilian_restaurant = folium.Map(location=[lat_toronto, lng_toronto], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(restaurant_br_df['Latitude'], 
                                           restaurant_br_df['Longitude'],
                                           restaurant_br_df['Borough'], 
                                           restaurant_br_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brazilian_restaurant)  
    
map_brazilian_restaurant

In [32]:
# 3 Restaurantes não se mostram suficientes para definirmos a localização, iremos localizar então outros empreendimentos de brasileiros

search_query = 'Brazilian'
radius = 10000
print(search_query + ' .... OK!')

Brazilian .... OK!


In [33]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, lat_toronto, lng_toronto, VERSION, search_query, radius)
url

'https://api.foursquare.com/v2/venues/search?client_id=KV1ZTVX03RWE1METG3FCOKUL2BOVNTMGK5CHOD5TOTH1CUSR&client_secret=HZDYU2YPKXSASI4N1J2ZUUVLODQF4FT2XSMY1Z5NXHV0BZLH&ll=43.653963,-79.387207&v=20180605&query=Brazilian&radius=10000'

In [34]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e1ba2d5949393001b8827fc'},
 'response': {'venues': [{'id': '52ec621e498ec68fa15ee922',
    'name': 'Copacabana Grilled Brazilian',
    'location': {'address': '230 Adelaide St W #2',
     'crossStreet': 'at Duncan St.',
     'lat': 43.64833259480477,
     'lng': -79.3881514766071,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.64833259480477,
       'lng': -79.3881514766071}],
     'distance': 631,
     'postalCode': 'M5H 3H1',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['230 Adelaide St W #2 (at Duncan St.)',
      'Toronto ON M5H 3H1',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d16b941735',
      'name': 'Brazilian Restaurant',
      'pluralName': 'Brazilian Restaurants',
      'shortName': 'Brazilian',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/argentinian_',
       'suffix': '.png'},
      'primary': True}],
    'referr

In [35]:
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d16b941735', 'name': 'B...",False,52ec621e498ec68fa15ee922,230 Adelaide St W #2,CA,Toronto,Canada,at Duncan St.,631,"[230 Adelaide St W #2 (at Duncan St.), Toronto...","[{'label': 'display', 'lat': 43.64833259480477...",43.648333,-79.388151,M5H 3H1,ON,Copacabana Grilled Brazilian,v-1578869460,NaN
1,"[{'id': '4bf58dd8d48988d1f1931735', 'name': 'G...",False,50452878e4b009b6ac03b849,NaN,CA,Toronto,Canada,NaN,348,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65268495156529...",43.652685,-79.383262,NaN,ON,Brazilian Day Toronto,v-1578869460,NaN
2,"[{'id': '52939a643cf9994f4e043a33', 'name': 'C...",False,4caf9fcecbab236a0e949973,95 Danforth Ave,CA,Toronto,Canada,at Broadview Ave,3413,"[95 Danforth Ave (at Broadview Ave), Toronto O...","[{'label': 'display', 'lat': 43.67630952807762...",43.676310,-79.358188,M4K 1N2,ON,Rodeo Brazilian Steakhouse,v-1578869460,NaN
3,"[{'id': '4bf58dd8d48988d110951735', 'name': 'S...",False,4e7f828b9a52e7965d12bebe,15 Scadding Avenue,CA,Toronto,Canada,NaN,1698,"[15 Scadding Avenue, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.64809061708176...",43.648091,-79.367739,NaN,ON,Brazilian Blowouts Toronto (BBTO),v-1578869460,NaN
4,"[{'id': '4bf58dd8d48988d108951735', 'name': 'W...",False,52af625411d21538524bfb43,486 Queen St W,CA,Toronto,Canada,at Spadina Ave,1284,"[486 Queen St W (at Spadina Ave), Toronto ON M...","[{'label': 'display', 'lat': 43.6478144790543,...",43.647814,-79.400704,M5V 2B3,ON,Lumieres de baia - Brazilian Bikinis & Fashion...,v-1578869460,79429016


In [36]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

#dataframe_filtered.shape
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Copacabana Grilled Brazilian,Brazilian Restaurant,230 Adelaide St W #2,CA,Toronto,Canada,at Duncan St.,631,"[230 Adelaide St W #2 (at Duncan St.), Toronto...","[{'label': 'display', 'lat': 43.64833259480477...",43.648333,-79.388151,M5H 3H1,ON,52ec621e498ec68fa15ee922
1,Brazilian Day Toronto,General Entertainment,NaN,CA,Toronto,Canada,NaN,348,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65268495156529...",43.652685,-79.383262,NaN,ON,50452878e4b009b6ac03b849
2,Rodeo Brazilian Steakhouse,Churrascaria,95 Danforth Ave,CA,Toronto,Canada,at Broadview Ave,3413,"[95 Danforth Ave (at Broadview Ave), Toronto O...","[{'label': 'display', 'lat': 43.67630952807762...",43.676310,-79.358188,M4K 1N2,ON,4caf9fcecbab236a0e949973
3,Brazilian Blowouts Toronto (BBTO),Salon / Barbershop,15 Scadding Avenue,CA,Toronto,Canada,NaN,1698,"[15 Scadding Avenue, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.64809061708176...",43.648091,-79.367739,NaN,ON,4e7f828b9a52e7965d12bebe
4,Lumieres de baia - Brazilian Bikinis & Fashion...,Women's Store,486 Queen St W,CA,Toronto,Canada,at Spadina Ave,1284,"[486 Queen St W (at Spadina Ave), Toronto ON M...","[{'label': 'display', 'lat': 43.6478144790543,...",43.647814,-79.400704,M5V 2B3,ON,52af625411d21538524bfb43
5,Radeo Brazilian Steakhouse,None,95 danforth av,CA,NaN,Canada,NaN,1854,"[95 danforth av, Canada]","[{'label': 'display', 'lat': 43.667045, 'lng':...",43.667045,-79.401458,NaN,NaN,4d183a7f25cda1437d597dd6
6,Brazilian Hair Studio,Health & Beauty Service,364 Oakwood Ave,CA,Toronto,Canada,Rogers Rd,5578,"[364 Oakwood Ave (Rogers Rd), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.68731307983398...",43.687313,-79.438911,NaN,ON,54ff2155498e94585e384e56
7,Braziliano's,None,NaN,CA,NaN,Canada,NaN,2056,[Canada],"[{'label': 'display', 'lat': 43.648788, 'lng':...",43.648788,-79.411714,NaN,NaN,4f3d476de4b022e41dad036f
8,Brazilian Adventist Church,Church,122 Mimico Ave,CA,Toronto,Canada,Royal York,9805,"[122 Mimico Ave (Royal York), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.61284182815670...",43.612842,-79.494828,NaN,ON,4e62a5d4d22d509a39ba31d6
9,Consulate General of Brazil In Toronto,Embassy / Consulate,77 Bloor St. West,CA,Toronto,Canada,at Bay St,1727,"[77 Bloor St. West (at Bay St), Toronto ON, Ca...","[{'label': 'display', 'lat': 43.66938936989137...",43.669389,-79.389584,NaN,ON,4e4f26c97d8bd425e48c95c2


In [37]:
print('There are {} uniques categories.'.format(len(dataframe_filtered['categories'].unique())))
dataframe_filtered['categories'].unique()

There are 9 uniques categories.


array(['Brazilian Restaurant', 'General Entertainment', 'Churrascaria',
       'Salon / Barbershop', "Women's Store", None,
       'Health & Beauty Service', 'Church', 'Embassy / Consulate'],
      dtype=object)

In [38]:
brazilian_venues = dataframe_filtered.filter(['postalCode','name','categories','lat','lng'], axis=1)
brazilian_venues.rename(columns={'name':'Venue'})

,postalCode,Venue,categories,lat,lng
0,M5H 3H1,Copacabana Grilled Brazilian,Brazilian Restaurant,43.648333,-79.388151
1,NaN,Brazilian Day Toronto,General Entertainment,43.652685,-79.383262
2,M4K 1N2,Rodeo Brazilian Steakhouse,Churrascaria,43.676310,-79.358188
3,NaN,Brazilian Blowouts Toronto (BBTO),Salon / Barbershop,43.648091,-79.367739
4,M5V 2B3,Lumieres de baia - Brazilian Bikinis & Fashion...,Women's Store,43.647814,-79.400704
5,NaN,Radeo Brazilian Steakhouse,None,43.667045,-79.401458
6,NaN,Brazilian Hair Studio,Health & Beauty Service,43.687313,-79.438911
7,NaN,Braziliano's,None,43.648788,-79.411714
8,NaN,Brazilian Adventist Church,Church,43.612842,-79.494828
9,NaN,Consulate General of Brazil In Toronto,Embassy / Consulate,43.669389,-79.389584


In [39]:
# instantiate a feature group for the incidents in the dataframe
venues = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(brazilian_venues['lat'],brazilian_venues['lng']):
    venues.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='yellow',
            fill_opacity=0.7
        )
    )

# add incidents to map
map_brazilian_restaurant.add_child(venues)

In [40]:
# Com isso vemos bairros onde possuem empreendimentos para brasileiros mas não tem restaurantes brasileiros
# Para finalizar vamos verificar onde estão as steakhouses que são empreendimentos muito apreciado por brasileiros

In [41]:
len(to_grouped[to_grouped['Steakhouse'] > 0])

36

In [42]:
to_steakhouse = to_grouped[['Neighborhood','Steakhouse']]
to_steakhouse

,Neighborhood,Steakhouse
0,"Adelaide, King, Richmond",0.100000
1,Agincourt,0.033333
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.033333
4,"Alderwood, Long Branch",0.033333
5,"Bathurst Manor, Downsview North, Wilson Heights",0.033333
6,Bayview Village,0.033333
7,"Bedford Park, Lawrence Manor East",0.000000
8,Berczy Park,0.033333
9,"Birch Cliff, Cliffside West",0.000000


In [43]:
to_merged = pd.merge(df_neighborhoods_coordinates, to_steakhouse, on='Neighborhood')
to_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Steakhouse
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.000000
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.000000
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.000000
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,0.000000
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0.000000
5,M9A,Queen's Park,Queen's Park,43.667856,-79.532242,0.000000
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0.033333
7,M3B,North York,Don Mills North,43.745906,-79.352188,0.033333
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,0.000000
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0.033333


In [44]:
# Pegar apenas bairros que tenham restaurantes Brasileiros               
df_rem = to_merged.loc[(to_merged['Steakhouse'] <= 0)]
steakhouse_df = to_merged.drop(df_rem.index)
steakhouse_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Steakhouse
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0.033333
7,M3B,North York,Don Mills North,43.745906,-79.352188,0.033333
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0.033333
18,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.033333
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0.033333
22,M1G,Scarborough,Woburn,43.770992,-79.216917,0.033333
26,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.066667
28,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,-79.442259,0.033333
30,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,0.100000
38,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,0.033333


In [45]:
# instantiate a feature group for the incidents in the dataframe
steakhouse = folium.map.FeatureGroup()

for lat, lng, in zip(steakhouse_df['Latitude'],steakhouse_df['Longitude']):
    steakhouse.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.7
        )
    )

# add incidents to map
map_brazilian_restaurant.add_child(steakhouse)

In [46]:
len(to_grouped[to_grouped["Steakhouse"] > 0])

36

In [47]:
to_br = to_grouped[["Neighborhood","Steakhouse","Brazilian Restaurant"]]

In [48]:
to_br.head(9)

,Neighborhood,Steakhouse,Brazilian Restaurant
0,"Adelaide, King, Richmond",0.100000,0.0
1,Agincourt,0.033333,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.033333,0.0
4,"Alderwood, Long Branch",0.033333,0.0
5,"Bathurst Manor, Downsview North, Wilson Heights",0.033333,0.0
6,Bayview Village,0.033333,0.0
7,"Bedford Park, Lawrence Manor East",0.000000,0.0
8,Berczy Park,0.033333,0.0


In [50]:
from sklearn.cluster import KMeans

kclusters = 3
to_clustering = to_br.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(to_clustering)
kmeans.labels_

array([0, 2, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 0, 1, 1, 1,
       2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 2, 0, 2, 2, 2, 1, 2, 1,
       0, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2], dtype=int32)

In [51]:
# add clustering labels
to_br.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = df_neighborhoods_coordinates
toronto_merged = toronto_merged.join(to_br.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.dropna(subset=["Cluster Labels"], axis=0, inplace=True)
toronto_merged.reset_index(drop=True, inplace=True)
toronto_merged['Cluster Labels'].astype(int)
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Steakhouse,Brazilian Restaurant
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,0.0,0.0
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,0.0,0.0
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1,0.0,0.0
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,1,0.0,0.0
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,1,0.0,0.0


In [52]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[lat_toronto, lng_toronto], zoom_start=11, width='90%', height='70%')

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

In [53]:
#Cluster 0
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Steakhouse,Brazilian Restaurant
26,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,0.066667,0.0
30,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,0,0.100000,0.0
50,M9L,North York,Humber Summit,43.756303,-79.565963,0,0.066667,0.0
60,M3N,North York,Downsview Northwest,43.761631,-79.520999,0,0.066667,0.0
77,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724,0,0.100000,0.0
94,M9W,Etobicoke,Northwest,43.706748,-79.594054,0,0.133333,0.0
97,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,0,0.066667,0.0


In [54]:
#Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Steakhouse,Brazilian Restaurant
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,0.0,0.000000
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,0.0,0.000000
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1,0.0,0.000000
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,1,0.0,0.000000
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,1,0.0,0.000000
5,M9A,Queen's Park,Queen's Park,43.667856,-79.532242,1,0.0,0.000000
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,1,0.0,0.000000
10,M6B,North York,Glencairn,43.709577,-79.445073,1,0.0,0.000000
11,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ...",43.650943,-79.554724,1,0.0,0.000000
12,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,1,0.0,0.000000


In [55]:
#Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Steakhouse,Brazilian Restaurant
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2,0.033333,0.0
7,M3B,North York,Don Mills North,43.745906,-79.352188,2,0.033333,0.0
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,2,0.033333,0.0
18,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2,0.033333,0.0
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,0.033333,0.0
22,M1G,Scarborough,Woburn,43.770992,-79.216917,2,0.033333,0.0
28,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,-79.442259,2,0.033333,0.0
38,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,2,0.033333,0.0
39,M2K,North York,Bayview Village,43.786947,-79.385975,2,0.033333,0.0
40,M3K,North York,"CFB Toronto, Downsview East",43.737473,-79.464763,2,0.033333,0.0


OBSERVATION

All the Brazilian restaurants are in cluster 1 which is around: Caledonia-Fairbanks, Dovercourt Village, Dufferin, Del Ray, Keelesdale, Mount Dennis and Silverthornand. Due the fact of there are only 3 Brazilian Restaurants in Toronto, open a new one in the region between Davenport, Harbord Village, Dufferin Grove, Yorkvile it is a good opportunity, because is near to region where brazilian people often visit. In this area there is no Brazilian restaurant, therefore, this project recommends the entrepreneur open an authentic Brazilian Restaurant around of these locations.